In [63]:
# some example data
import pandas as pd
from typing import Union
from statsmodels.tsa.vector_ar.var_model import VARResultsWrapper
from statsmodels.tsa.api import VAR

from statsmodels.tsa.base.datetools import dates_from_str

In [13]:
"""
要把 dataframe 的格式清理成以下格式
dates_from_str 要放入 index
剩下的資料只能保留 feature
"""
# statsmodels的範例資料
import statsmodels.api as sm
mdata = sm.datasets.macrodata.load_pandas().data
mdata
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)
mdata.index = pd.DatetimeIndex(quarterly)
data = mdata.drop(columns=["year","quarter"])
data

# KAGGLE 範例資料 1 
dir = "C:/Users/foresight_User/Desktop/"
data = pd.read_csv(dir+"National Stock Exchange/infy_stock.csv")
data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
data = data.drop(columns=["Date","Symbol","Series"])
data

# KAGGLE 範例資料 2
data = pd.read_csv(dir+"National Stock Exchange/nifty_it_index.csv")
data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
data = data.drop(columns=["Date"])
data

# KAGGLE 範例資料 3
data = pd.read_csv(dir+"National Stock Exchange/tcs_stock.csv")
data.index =pd.DatetimeIndex(dates_from_str(data["Date"]))
data = data.drop(columns=["Date","Symbol","Series"])
data

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
1959-03-31,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1959-06-30,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
1959-09-30,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
1959-12-31,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
1960-03-31,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...
2008-09-30,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
2008-12-31,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
2009-03-31,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
2009-06-30,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [56]:
def cal_maxLag(data:pd.DataFrame):
    n_totobs = len(data)
    ntrend = 1 #len(trend) if trend.startswith("c") else 0
    neqs = data.shape[1]
    max_estimable = (n_totobs - neqs - ntrend) // (1 + neqs)
    return max_estimable

In [58]:

def vectorAutoregression(data:pd.DataFrame,maxlags:Union[int,str]="auto",ic:str=None):
    model = VAR(data)
    # ==== 這邊不要動 =====

    """ 
    這是套件設定的
    trend : str {"n", "c", "ct", "ctt"}
        * "n" - no deterministic terms
        * "c" - constant term
        * "ct" - constant and linear term
        * "ctt" - constant, linear, and quadratic term

    maxlags 不可以超過 max_estimable 的值
    maxlags 為模型擬合最大數值
    statemodel有設定條件，已經寫在下述的程式
    使用者要調整低於 maxlags
    """
    max_estimable = cal_maxLag(data)
    # ==== 這邊不要動 =====

    print(" maxlags 要小於等於: ", max_estimable)

    if maxlags == "auto":
        maxlags = max_estimable
    if maxlags > max_estimable:
        raise Exception(" maxlags 要小於等於: ", max_estimable)


    """
    ic 為評估模型的好壞
    ic = {'aic', 'fpe', 'hqic', 'bic', None}
    Information criterion to use for VAR order selection.
    aic : Akaike
    fpe : Final prediction error
    hqic : Hannan-Quinn
    bic : Bayesian a.k.a. Schwarz
    """

    results = model.fit(maxlags=maxlags, ic=ic)
    print(f"在最大 lag 數目為 {max_estimable} 的情況下，VAR 找出的最佳 lag 為: ",results.k_ar)
    return results

In [59]:
def vectorAutoregressionRelationship(results:VARResultsWrapper,target:str,pvalue_threshold:float=0.05):
    # target  客人關心的 Y 是甚麼，Y 會包含在 results 中

    coef_df = results.params[target]
    pvalues_df = results.pvalues[target]

    # 合并系数和p值
    summary = pd.concat([coef_df, pvalues_df], axis=1)
    summary.columns = ['coef', 'pvalue']
    summary = summary.drop(index="const").reset_index()
    summary_index = summary["index"].str.split(".", expand=True).rename(columns={0:"time_lag",1:"feature"})
    summary_index["time_lag"] = summary_index["time_lag"].str.replace("L","").astype(int)
    summary = pd.concat([summary_index,summary],axis=1).drop(columns="index")
    summary = summary[summary["pvalue"]<pvalue_threshold].reset_index(drop=True)
    return summary

In [64]:
"""
讓使用者決定好奇的target是甚麼，套件會自動找出所有跟target相關的不同time lag的時間變相
maxlags 會影響最終最佳推薦的結果，也得慎選，但設定上不可以超過 cal_maxLag 輸出的值
"""
maxlags = cal_maxLag(data) # USER 不能指定超過這個的LAG，LAG值也要大於0
target = "Trades"
results = vectorAutoregression(data,maxlags=maxlags,ic="fpe")
pvalue_threshold = 0.05
VAR_relationship = vectorAutoregressionRelationship(results=results,target=target,pvalue_threshold=pvalue_threshold)

 maxlags 要小於等於:  18
在最大 lag 數目為 18 的情況下，VAR 找出的最佳 lag 為:  1


c:\Users\foresight_User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [61]:
VAR_relationship

,time_lag,feature,coef,pvalue
0,1,Trades,0.553129,2.080915e-09


In [62]:
"""
生成VAR關係公式
"""
"""
可有可無，看需求
"""
formula_parts = []
for index, row in VAR_relationship.iterrows():
    coef = row['coef']
    if coef < 0:
        term = f"- {-coef:.3f} * {row['feature']}(t-{row['time_lag']})"
    else:
        term = f"{coef:.3f} * {row['feature']}(t-{row['time_lag']})"
    formula_parts.append(term)

formula = " + ".join(formula_parts).replace("+ -", "- ")
target_formula = f"f{target}(t) = {formula}"

# 打印公式
print(target_formula)

fTrades(t) = 0.553 * Trades(t-1)
